# 2017-05-28 : working on the video processing
---
I need find a way to bypass the *fl_image()* function limitations.
It's easy with MoviePy to retrieve the video frame rate :

    print(video.fps)
    print(processed_video.fps)
    ==========================
    25.0
    25.0
    
I'll try to build the script into a class, to save parameters between each frame iteration. Or use openCV again (but we try to use it as little as possible to avoid compiling as mush as possible, and MoviePy is only composed of python scripts, contrary to openCV that contain a lot of C scripts).  
I need to integrate : recording of a video file by the Pi, sending that file -frame by frame- to server upon request, that may (or may not) do some processing.  

When moving from the use of a *processing* function called by the *fl_image()* command to the use of :

    for frame in video.get_frame(t=0.0 % video.duration):
        (...)
        
I successfuly process the video file. When asked, the script also output 480 frames, no matter the value of *t*.  But then the frame is an numpy.array not callable by :

    processed_video.fl_image(frame)
    =================================================
    TypeError: 'numpy.ndarray' object is not callable
    
I remember getting through that error with the *matplotlib* module. Or I may use PIL (as suggested here : https://stackoverflow.com/questions/10965417/how-to-convert-numpy-array-to-pil-image-applying-matplotlib-colormap) and its *frombytes* function (more informations : http://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.fromarray).  

But when adding the first line : 

    foo = Image.fromarray(frame)
    processed_video.fl_image(foo)
    =========================================
    TypeError: 'Image' object is not callable
    
What I need to do is convert the data from numpy array to RGB to be readable to *fl_image*.  
I'll try one of the methods proposed here : http://www.socouldanyone.com/2013/03/converting-grayscale-to-rgb-with-numpy.html  
Yet, what is proposed looks like what's already written in my scripts, e.g : 

    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, :] = im[:, :, np.newaxis]
    
Looks similar to : 

    frame = np.zeros((h, w, 3), dtype = np.uint8)
    frame[:, :, 0][np.unravel_index(data[:n_datapoints], (h, w))] = 255
    frame[:, :, -1][np.unravel_index(data[-n_datapoints:], (h, w))] = 255
    
So the data is arleady a RBG data, the problem may come from the way I call *fl_image()*. That function first call the *get_frame()* one (see : https://zulko.github.io/moviepy/ref/VideoClip/VideoClip.html#moviepy.video.VideoClip.VideoClip.fl_image), but I feed it with a numpy array. I need to find a way to grab the frames times.  

I tried another methode following these : https://gist.github.com/nirum/d4224ad3cd0d71bfef6eba8f3d6ffd59 and https://stackoverflow.com/questions/7200878/python-list-of-np-arrays-to-array, wrote the following lines, that start to work fine but then output an error :

        frames = np.stack(frame)
        (...)
    processed_video = processed_video.write_images_sequence(frames)
    ===============================================================
    [MoviePy] Writing frames [[[255   0 255]
      [255   0 255]
      [255   0 255]
      (...)
    (...)
    OSError: Cannot understand given URI: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    (...)

# 2017-05-29
---
What I don't understand about that error, is that URI are supposed to be part of networks (see : https://fr.wikipedia.org/wiki/Uniform_Resource_Identifier), but I'm doing a local processing...  
The way that error is called is interesting : 

    ---> 38     processed_video = video.write_images_sequence(frame)
    (...)
    214             if len(uri_r) > 60:
    215                 uri_r = uri_r[:57] + '...'
    --> 216             raise IOError("Cannot understand given URI: %s." % uri_r)
    
So the data length may be involved.  

Since I have only a few days of intership, I have to stop working on my scripts and start doing so on my poster right now.  
To do it, I'll use the document preparation system called LaTeX and the free cross-platform editor Texmaker (more informations : http://www.xm1math.net/texmaker/index.html).  
L. Perrinet had the kindness to provide the framework he's using for his own posters to help me speed up the process.  

I first had to install a LaTeX distribution called TeX Live : 

    sudo aptitude install texlive
    
It contain the main libraries. I could have installed a bigger version called *texlive-full* but it was more than 8Gb and I didn't need so many libraries.  

TexMaker offert two versions : an i386 package and an amd64 package. To find which one my computer is using (as proposed here : https://ubuntuforums.org/showthread.php?t=1250624) :

    uname -m
    =========
    x86_64
    
So I need to get the amd64 one. I downloaded the corresponding .deb package (available here : http://www.xm1math.net/texmaker/download.html) and installed it through Ubutuntu's Software Manager.  

I then opened the program and L.Perrinet's framework, but when I try to convert it to pdf, it output an error telling me that the *dvipsnames* package is missing. To override it, I finally installed the *texlive-full* version but that made me lost almost an hour...  
But even after that, some of the used packages are still missing (*asmaths*,*multicol*, ...).

I first tried to update the installed packages, in case the ones I look for were are present but outdated :

    tlmgr update --all
    =====================================================================
    Cannot determine type of tlpdb from /home/albert/texmf!
    cannot setup TLPDB in /home/albert/texmf at /usr/bin/tlmgr line 5713.
    
tlmgr is the command name of the TeX Live package manager (more informations : https://www.tug.org/texlive/pkginstall.html). I had to initialize it to work properly :

    tlmgr init-usertree
    =======================================================
    Cannot determine type of tlpdb from /home/albert/texmf!
    
Then :

    tlmgr update --all
    ==================================================================
    /usr/bin/tlmgr: Initialization failed (in setup_unix_one):
    /usr/bin/tlmgr: could not find a usable xzdec.
    /usr/bin/tlmgr: Please install xzdec and try again.
    tlmgr: Couldn't set up the necessary programs.
    Installation of packages is not supported.
    Please report to texlive@tug.org.
    Use of uninitialized value r in split at /usr/bin/tlmgr line 3899.
    tlmgr: Cannot find main repository, you have to tag one as main!

So I installed xzdec, that is a tiny compressor/decompressor utility (as suggested here : https://tex.stackexchange.com/questions/137428/tlmgr-cannot-setup-tlpdb) : 

    sudo aptitude install xzdec
    
Then :

    tlmgr update --all
    ===========================================================================================
    (running on Debian, switching to user mode!)
    TeX Live 2016 is frozen forever and will no
    longer be updated.  This happens in preparation for a new release.

    If you're interested in helping to pretest the new release (when
    pretests are available), please read http://tug.org/texlive/pretest.html.
    Otherwise, just wait, and the new release will be ready in due time.
    tlmgr: package repository http://mirrors.ircam.fr/pub/CTAN/systems/texlive/tlnet (verified)
    tlmgr: no updates available

I tried to install one of the missing package :

    tlmgr install multicol
    ======================
    (...)
    tlmgr: package repository http://mirrors.standaloneinstaller.com/ctan/systems/texlive/tlnet (verified)
    tlmgr install: package multicol not present in repository.

After several hours of research, I asked L. Perrinet for a solution and he led me to the following function (with that webpage: https://askubuntu.com/questions/481/how-do-i-find-the-package-that-provides-a-file
), that helped me found out the *multicol* package seems to be already installed : 

    dpkg -S  multicol
    =================================
    (...)
    texlive-latex-base-doc: /usr/share/doc/texlive-doc/latex/tools/multicol.pdf
    texlive-lang-arabic: /usr/share/texlive/texmf-dist/tex/xelatex/bidi/multicol-xetex-bidi.def
    texlive-latex-extra-doc: /usr/share/doc/texlive-doc/latex/adjmulticol/adjmulticol.pdf
    texlive-latex-extra: /usr/share/texlive/texmf-dist/tex/latex/adjmulticol
    texlive-latex-extra-doc: /usr/share/doc/texlive-doc/latex/adjmulticol/sample.pdf
    texlive-htmlxml: /usr/share/texlive/texmf-dist/tex/generic/tex4ht/multicol.4ht
    texlive-latex-base: /usr/share/texlive/texmf-dist/tex/latex/tools/multicol.sty
    (...)

So the problem may come from the pathway that Texmaker uses to look for packages.  
After uninstalling/reinstalling TexMaker to see if it would change anything, I noticed that lot of packages, even "native" ones, outputed an error when called. I deleted them from the file one by one until I understood that the one that was missing was actually :

    \usepackage[dvipsnames]{x}
    
When putting it as a comment, I don't get the fatal errors anymore.  
Now lot of errors are still outputed (almost one hundred!) but I'll be able to solve them one by one.  

I easily solved some of them by removing mentions to a third author.  
Now majority of errors are caused when the script calls the xcolor package, e.g :

    ! Package xcolor Error: Undefined color 'MidnightBlue'. (...)
    (...)

Removing these colors cancel related errors. I may add them later (see : http://mirrors.ircam.fr/pub/CTAN/macros/latex/contrib/xcolor/xcolor.pdf) but I need a functionnal script as soon as possible.  

As suggested by L. Perrinet, I may use the Tikz package (see: http://www.texample.net/tikz/) to draw figures.

# Outline
---
- [ ] Introduce comparaisons between frames to only code moving pixels
    - [X] Solve an issue : script correctly modifying only the first frame
    - [X] Solve an issue : "write file" command don't contain modifications
    - [ ] Solve an issue : incomplete processing
        - [X] Find a way to substract frame with the previous one
        - [X] Convert the numpy array data to RGB data
- [X] Convert the video file to mp4 instead of avi
- [X] Capture a video with a stable background and moving object
- [ ] Implement the scripts into OpenRetina
- [ ] Make the Pi do the GC layer work 
- [ ] [OPT] Add a way to delete the video file on the Pi after sending and processing are done.
- [ ] [OPT] Try to find the optimal chunk size for a faster file transfer.
- [ ] [OPT] Rewrite the scripts to introduce classes
- [ ] [OPT] Dive into the video processing steps to find out when the sixth second (if the video is 5s long) is added

Going further :
- [ ] Study the program bottlenecks (Where does the program lose computing time?)
    - [ ] Add flags to monitors each script
- [ ] Switch from synchronous (phrs grabbing and sending frames when receiving other layers requests) to asynchronous (phrs grabbing continously and sending frames when requested) processes
- [ ] Add a way to use external videos (e.g from Youtube) as a video input

Poster :
- [X] Install LaTeX
- [X] Install TexMaker
- [X] Solve an issue : install the missing packages
    - [CANCELED] dvipsnames
    - [CANCELED] multicol
- [X] Solve an issue : delete everything related to a third author
- [X] Solve an issue : xcolor undefinied colors
- [X] Write poster outline
- [ ] Find a title
- [ ] Write abstract
- [ ] Take pictures for each section